In [111]:
from selenium import webdriver
from PIL import Image
from io import BytesIO
import requests
from bs4 import BeautifulSoup
import os
import time

In [34]:
%%time
verbose = 1
keyword = '맨투맨'
male_cat_id = 50000830
female_cat_id = 50000803
page = 1
male_shop_url = f'https://search.shopping.naver.com/search/all.nhn?origQuery={keyword}&pagingIndex={page}&pagingSize=40&viewType=list&sort=rel&cat_id={male_cat_id}&frm=NVSHCAT&query=%EB%A7%A8%ED%88%AC%EB%A7%A8'
female_shop_url = f'https://search.shopping.naver.com/search/all.nhn?origQuery={keyword}&pagingIndex={page}&pagingSize=40&viewType=list&sort=rel&cat_id={female_cat_id}&frm=NVSHCAT&query=%EB%A7%A8%ED%88%AC%EB%A7%A8'

driver_path = './geckodriver.exe'
options = webdriver.FirefoxOptions()

#options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko')

browser = webdriver.Firefox(executable_path=driver_path, options=options)
browser.get(male_shop_url)
# 페이지 가지고올 수 있게 최대 15초까지 기다림 
browser.implicitly_wait(15)

Wall time: 6.2 s


In [35]:
%%time
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

Wall time: 554 ms


In [36]:
goods_list = soup.select_one('ul.goods_list').find_all('div', {'class':'info'})

In [37]:
len(goods_list)

44

In [60]:
for i in range(15):
    many_shop = goods_list[i].find('span', {'class':'price'}).find('a')
    if i%5 == 0:
        print(f'{i}...')
    if many_shop == None:
        shop_title_list.append(goods_list[i].find('a').attrs['title'])
        shop_href_list.append(goods_list[i].find('a').attrs['href'])
    else:
        if '판매처' in many_shop.text:
            shop_title_list.append(goods_list[i].find('a').attrs['title'])
            shop_detail = many_shop.attrs['href']
            #print(shop_detail)
            browser.get(shop_detail)
            browser.implicitly_wait(15)
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            shop_href_list.append(soup.select_one('div.price_area').find('a').attrs['href'])
        else:
            shop_title_list.append(goods_list[i].find('a').attrs['title'])
            shop_href_list.append(goods_list[i].find('a').attrs['href'])

단체 맨투맨 아동용 성인용 쮸리 단체맨투맨
https://adcr.naver.com/adcr?x=8lXuGmodWfzIgISwZ7d4s////w==krZvHSi6MB2DhdBbPVGzJzPEYzYv6z2gUkkSkiEnRZIcFDpZO1s25CWtAB25dBmMSRABtN/fl2Ap29xvPwWbUu8ikHs1iNKjItWJRM0kV1xmovNG3d26WZIlpZp3KAlOa6d1/ilZQz9pGphgnGwVta/njXALWLaCDo5UC9Cf3U75XFPzb9w3TABYMCK8mgPpFO2K/UGer5NH3noDBvx4zvBtpxvndr6AqdqUbZK2rqtgBciyCCBIyCMYqmYkXnjQSIvJK6sLfDSzDWs3tWncmtLpvcm7f615EyqzqNU1T19KSAsbQvOfZIlJX4yoXvgApZtqNtTk8W/MzCFss8kvq9G1T/nJtH/APpw1ePrGbeyj72toxAQcGVED90aos5HgBYdQLPzkT9lLMG9oBsIKCzx3Pk/z6ZCMmNxwIfqmp+u8rP6Xe1vyKBkgEfUO75HwEb7Xdlp/G9Ig4AFaG6JsSpquK43LOOynX2gWQqxCJ0f0RA9BxzptHxpFD7ellwI8J8Cp2UnZetttQt9Pz9azkV6dcvDRiUbMHzJua8SVAN+ntiK8oBXCQzKoI/QQ+NwGLbcGbWLdeTBvIzbp9UFlaAjuNm8o8I5NKzKPZeeAtai2eEjm3n0qjGHpQfNg7YUpYWG8uz9RMlYg57HfXxJrgx8GhdwUd7p6V6lQm20IrERruXzhXBo2cbT4orm2rpYvQuN1KLEPRwImHTMzucHnKLhB+02gvoNjfcPcMv+4dbQdUZ/+LQJYabMThrK7FjIqVfDIyg6Euoe9ByAZyuzlu2AvUp/rNoljLd8oMHTYYxo2N0ndv+cEppWtbRTgo0lz4Qn5Ihck2oit2lSxzT8Xb0Anj2N/H8kLV07Re+hyGCRU7zJr5aXCBJzNZ0/vIN6xRKQ3DmDzNLb3o1TEFEXjIZ

http://cr2.shopping.naver.com/adcrNoti.nhn?x=VDrbcV2KOEVM2KLctXUP1%2F%2F%2F%2Fw%3D%3DsIwdEdInRTidir49%2FT2MbnqAKFYfkrjgv3tK6H%2Be9hw0zGGvRbI53cXMFK5xhei7zJbo0NSP6%2BUGG2ppjD93lyOiyTJeJUOv2TiA8o%2FUxxt4AJ48p1gvSf%2B49sywr%2BMbY9vhUHbiP3PsE1qIx7Xtt%2BRl6Skqa8Vk%2FNRlFdVPNkScR7W5gyJQ1nvFY%2BMtklQrrn8qD10eVt1lGhKzM%2BKR9ohJwRse4yLJ6KioKK%2FsNHIb%2FEX2JnxnuHg0Y9gw4Pky5G%2B1Yswqns5XOL7BW1357YuCwWIQErpecaB5%2BpIXqoHAZGil%2B6U717OlhtXtqAG1V7QJQmHpMXV27IknvclvA%2Fa9BpmK435PD9K1hykv2Il3D4WQ7Vf27UCXyouOtVYUo6403CrgW61qH1HQAqapI3Dqq7KMKtB9UY9ZX%2Fey1drcCw18zUc3oDgR6eLWX53rRyEBWTPe4nV5Wo44B5l%2FIZMa6Berw%2F0Lr17CX%2Fsic9XFNgyhNItaJhZw4GP0vaVHHYYuAd1oxkgVlpGf7%2FiKQpiUsAJ1s%2B23%2B9cBEEJ2rsJFSGZKhwds5yGiTGDy2XHKu4j349JoVGKvHl9ezQbk4uq0mbF1%2FpQrxHhQ24gIwl1j6gJOCKUniKRXxgU2aBZ2wdFHMOli47%2F0VU9v4dIW1BIRh6gel6%2B6JppsM4Ogv1qo%3D&nv_mid=18809629410&cat_id=50000830
--------------------------------------------------
특양면 베이직 맨투맨 기모 무지
http://cr2.shopping.naver.com/adcr.nhn?x=pKgYGcapr1h0krx8GQT8Mv%2F%2F%2

In [124]:
def browser_on():
    driver_path = './geckodriver.exe'
    options = webdriver.FirefoxOptions()

    #options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko')

    return webdriver.Firefox(executable_path=driver_path, options=options)

In [125]:
def href_crawler(browser, male, page):    
    keyword = '맨투맨'
    if male:
        cat_id = 50000830
    else:
        cat_id = 50000803
    shop_title_list = []
    shop_href_list = []
    
    for p in range(1,page+1):
        try:
            print(f"{p}page crawling...")
            main_url = f'https://search.shopping.naver.com/search/all.nhn?origQuery={keyword}&pagingIndex={p}&pagingSize=40&viewType=list&sort=rel&cat_id={male_cat_id}&frm=NVSHCAT&query=%EB%A7%A8%ED%88%AC%EB%A7%A8'
            browser.get(main_url)
            # 페이지 가지고올 수 있게 최대 15초까지 기다림 
            browser.implicitly_wait(15)
        
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            goods_list = soup.select_one('ul.goods_list').find_all('div', {'class':'info'})
        except:
            continue
            for i in range(len(goods_list)):
                try:
                    many_shop = goods_list[i].find('span', {'class':'price'}).find('a')
                    if i%5 == 0:
                        print(f'{i}...')
                    if many_shop == None:
                        shop_title_list.append(goods_list[i].find('a').attrs['title'])
                        shop_href_list.append(goods_list[i].find('a').attrs['href'])
                    else:
                        if '판매처' in many_shop.text:
                            shop_title_list.append(goods_list[i].find('a').attrs['title'])
                            shop_detail = many_shop.attrs['href']
                            #print(shop_detail)
                            browser.get(shop_detail)
                            browser.implicitly_wait(15)
                            html = browser.page_source
                            soup = BeautifulSoup(html, 'html.parser')
                            shop_href_list.append(soup.select_one('div.price_area').find('a').attrs['href'])
                        else:
                            shop_title_list.append(goods_list[i].find('a').attrs['title'])
                            shop_href_list.append(goods_list[i].find('a').attrs['href'])
                except:
                    continue
        
    return shop_title_list, shop_href_list

In [126]:
%%time
browser = browser_on()

Wall time: 5.16 s


In [127]:
%%time
male_title_list, male_href_list = href_crawler(browser,True, 100)
female_title_list, female_href_list = href_crawler(browser,False, 100)

1page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
2page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
3page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
4page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
5page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
6page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
7page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
8page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
9page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
10page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
11page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
12page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
13page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
14page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
15page crawling

NoSuchWindowException: Message: Browsing context has been discarded


In [131]:
print(len(male_href_list))
print(len(female_href_list))

NameError: name 'male_href_list' is not defined

In [129]:
def image_scrapping(browser,href_list, save_path):
    for idx, href in enumerate(href_list):
        print(f'{idx+1}/{len(href_list)} screent shot...')
        try:
            browser.get(href)
            browser.implicitly_wait(15)
            time.sleep(7)
            # from here http://stackoverflow.com/questions/1145850/how-to-get-height-of-entire-document-with-javascript
            js = 'return Math.max( document.body.scrollHeight, document.body.offsetHeight,  document.documentElement.clientHeight,  document.documentElement.scrollHeight,  document.documentElement.offsetHeight);'
            scrollheight = browser.execute_script(js)
            if scrollheight > 20000:
                print(f'scrollheight over: {scrollheight} !!')
                continue
            scale = 0.8
            browser.execute_script(f'document.body.style.MozTransform = "scale({scale})";')
            time.sleep(2)

            slices = []
            offset = 0
            while offset < scrollheight:
                browser.execute_script("window.scrollTo(0, %s);" % offset)
                time.sleep(2)
                img = Image.open(BytesIO(browser.get_screenshot_as_png()))
                offset += img.size[1]

                slices.append(img)

                browser.get_screenshot_as_file('%s/screen_%s.png' % ('/tmp', offset))
                # print (f'{offset} / {scrollheight}')


            screenshot = Image.new('RGB', (slices[0].size[0], scrollheight))
            offset = 0
            for img in slices:
                screenshot.paste(img, (0, offset))
                offset += img.size[1]
            screenshot.save(f'./img/{save_path}/screen_shot_{idx}.png')
        except:
            continue
            

In [130]:
image_scrapping(browser, male_href_list, 'male_size')
image_scrapping(browser, female_href_list, 'female_size')

NameError: name 'male_href_list' is not defined